<a href="https://colab.research.google.com/github/armandochernandez-ai/Curso-python-slava/blob/main/Unidad_3/proceso_admision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
from google.colab import drive
import pandas as pd
import sys

instruccion = """Este es un proceso de selección de los mejores aspitantes (puntajes) a ingresar a facultades.
Deberá elegir un cupo para cada facultad, con lo cual se generará una lista de los mejores aspirantes."""

print(instruccion)

# Montar Google Drive si no está montado
drive.mount('/content/drive')

# === 1. Cargar archivo texto a tabla simple ===
#archivo = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
#tabla = pd.read_table(archivo)
#tabla


# === 2. Convertir a tabla a dataframe ===
archivo = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
#se pueden definir columnas antes
n_columnas = ["Nombre","Apellido","ESP","EMA","EPH","ECH","ECS", "fac_1", "fac_2","fac_3"]
df = pd.read_csv(archivo, sep=' ', names = n_columnas)
df

#O cargar y poner Nombre a columnas
df = "/content/drive/MyDrive/Colab Data/applicant_list_7.txt"
df = pd.read_csv(df, sep=' ', header=None)
#df = pd.DataFrame(df)
df

#Colocar encabezados
df.columns = ["Nombre","Apellido","ESP","EMA","EPH","ECH","ECS", "fac_1", "fac_2","fac_3"]
df

# Crear el esquema para traducir al español
traduccion = {
    'Physics': 'Física',
    'Engineering': 'Ingeniería',
    'Mathematics': 'Matemáticas',
    'Biotech': 'Biotecnología',
    'Chemistry': 'Química'
}

# Aplicar la traduccióna las columnas que lo requiern
df['fac_1'] = df['fac_1'].map(traduccion)
df['fac_2'] = df['fac_2'].map(traduccion)
df['fac_3'] = df['fac_3'].map(traduccion)

# Presentar
df.head()

# Función para buscar columna
def buscar(df, facultad):
  """Búsqueda en data frame en la columna fac_1 para filtar despues."""
  return df[df['fac_1'].str.contains(facultad, case=False, na=False)]

# Pregunta
facultad = str(input("¿Que facultad busca? ")).lower()

try:
    # Búsqueda de resultados en columna
    resultados_busqueda = buscar(df, facultad)

    # Resultados posibles
    if resultados_busqueda.empty:
        # En caso de no encontrar facultad
        raise ValueError(f"'{facultad}'")
    else:
      print(f"Facultad encontrada: {facultad}")
      #Cuando se encuentra

except ValueError as no:
    # Reporte
    print(f"Facultad {no} no existe")
    #sys.exit()

except Exception as e:
    # Otros errores
    print(f"Error inesperado: {e}")

# Vamos a definir el cupo de alumnos
try:
    cupo = int(input(f"¿Cuántos cupos hay para {facultad}? "))
    if cupo < 0:
        raise ValueError("El número de cupos debe ser mayor que 0.")
except ValueError as e:
    print(f"Error: {e}")
    sys.exit()

print(f"Se han definido {cupo} cupos para {facultad}.")

def calc_promedio(row, facultad):
    """Calcular promedios por facultad."""
    if facultad in ['Física', 'Ingeniería', 'Matemáticas']:
        promedio = row[['ESP', 'EMA', 'EPH']]
    elif facultad in ['Química', 'Biotecnología']:
        promedio = row[['EMA', 'ECH', 'ECS']]
    else:
        return None  # Para caso de facultades no reconocidas

    return promedio.mean()

# Lista de facultades psibles
posibles = list(traduccion.values())

# Iteracion a las facultades psobles
for facultad_nombre in posibles:
  column_name = f'{facultad_nombre}_Promedio'
  df[column_name] = df.apply(calc_promedio, axis=1, facultad=facultad_nombre)

df.head()

# Normalizar resultados
df['fac_1_normalized'] = df['fac_1'].str.lower().str.strip()
df['fac_2_normalized'] = df['fac_2'].str.lower().str.strip()
df['fac_3_normalized'] = df['fac_3'].str.lower().str.strip()
df.head()

# Convertir textos de nombres de facultades a minusculas
facultad_lower = facultad.lower()

# Filtar para seleccionar de columna fac_1_normalizado igual a nombre minusculas
candidatos_fac1 = df[df['fac_1_normalized'] == facultad_lower].copy()

# Calcula promedio para cada candidadto de fac_1
candidatos_fac1['Promedio'] = candidatos_fac1.apply(calc_promedio, axis=1, facultad=facultad.capitalize())

# Ordena de forma ascendente por columna Promedio
candidatos_fac1 = candidatos_fac1.sort_values(by='Promedio', ascending=False)

# Selecciona los mejores de acuerdo al cupo
seleccion_fac1 = candidatos_fac1.head(cupo)

# Lista de fac_1
print(f"Lista admitidos a {facultad.capitalize()} como primera opción:")
display(seleccion_fac1[['Nombre', 'Apellido', 'fac_1', 'Promedio']])
print()

#Definir cupo remanente
cupo_disponible = cupo - len(seleccion_fac1)
print(f"Cupo disponible para {facultad.capitalize()} {cupo_disponible} lista de candidatos:")


#Definir candidatos a ocupar espacios disponibles de la segunda y tercera opción
candidatos = df[~df.index.isin(seleccion_fac1.index)].copy()
#Desplegar lista de candidatos
display(candidatos[['Nombre', 'Apellido', 'fac_2', 'fac_3']])


seleccion_otras = []

for index, row in candidatos.iterrows():
    candidatos_data = row.to_dict()
    candidatos_data['Promedio'] = None # Inicializar promedio

    if row['fac_2_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)
    elif row['fac_3_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)

# Convertir a DataFrame
seleccion_otras_df = pd.DataFrame(seleccion_otras)
print(f"Encabezados de lista convertida a DataFrame:")
display(seleccion_otras_df.head())

#Selección final
seleccion_otras_df = seleccion_otras_df.sort_values(by='Promedio', ascending=False)
candidatos_otras = min(cupo_disponible, len(seleccion_otras_df))
seleccion_final_otras = seleccion_otras_df.head(candidatos_otras)
estudiantes_admitidos = pd.concat([seleccion_fac1, seleccion_final_otras], ignore_index=True)

print(f"Lista final de estudiantes admitidos {facultad.capitalize()}:")
lista_final = estudiantes_admitidos.sort_values(by='Promedio', ascending=False)
display(lista_final[['Nombre', 'Apellido', 'fac_1', 'Promedio']])

Este es un proceso de selección de los mejores aspitantes (puntajes) a ingresar a facultades.
Deberá elegir un cupo para cada facultad, con lo cual se generará una lista de los mejores aspirantes.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
¿Que facultad busca? química
Facultad encontrada: química
¿Cuántos cupos hay para química? 25
Se han definido 25 cupos para química.
Lista admitidos a Química como primera opción:


,Nombre,Apellido,fac_1,Promedio
47,Kennedy,Barrett,Química,91.333333
81,Ayeshia,Jackman,Química,86.666667
62,Estephanie,Phelps,Química,82.000000
2,Uzma,Naysmythe,Química,81.666667
14,Franki,Dinnis,Química,81.666667
50,Crescentia,Dow,Química,75.000000
9,Delta,Fanny,Química,73.333333
75,Brittania,Denny,Química,72.333333
79,Stacey,Revill,Química,71.000000
74,Pearl,Pullins,Química,66.000000



Cupo disponible para Química 14 lista de candidatos:


,Nombre,Apellido,fac_2,fac_3
0,Jermine,Brunton,Ingeniería,Matemáticas
1,Justo,Mirfin,Biotecnología,Química
3,Koury,Wingo,Biotecnología,Matemáticas
4,Kentrell,Hillhouse,Ingeniería,Biotecnología
5,Trica,Macalpine,Matemáticas,Biotecnología
...,...,...,...,...
93,Tamkia,Fish,Ingeniería,Matemáticas
94,Deniz,Blanchard,Biotecnología,Química
95,Mira,Riley,Biotecnología,Matemáticas
96,Loura,Macansh,Física,Matemáticas


Encabezados de lista convertida a DataFrame:


,Nombre,Apellido,ESP,EMA,EPH,ECH,ECS,fac_1,fac_2,fac_3,Física_Promedio,Ingeniería_Promedio,Matemáticas_Promedio,Biotecnología_Promedio,Química_Promedio,fac_1_normalized,fac_2_normalized,fac_3_normalized,Promedio
0,Justo,Mirfin,71,77,61,60,41,Ingeniería,Biotecnología,Química,69.666667,69.666667,69.666667,59.333333,59.333333,ingeniería,biotecnología,química,59.333333
1,Laney,Braithwaite,90,90,90,72,94,Física,Química,Matemáticas,90.000000,90.000000,90.000000,85.333333,85.333333,física,química,matemáticas,85.333333
2,Spring,Burridge,71,84,98,71,50,Matemáticas,Química,Ingeniería,84.333333,84.333333,84.333333,68.333333,68.333333,matemáticas,química,ingeniería,68.333333
3,Elen,Ashbury,54,76,88,62,60,Matemáticas,Química,Biotecnología,72.666667,72.666667,72.666667,66.000000,66.000000,matemáticas,química,biotecnología,66.000000
4,Artavious,Fay,71,77,61,60,61,Ingeniería,Biotecnología,Química,69.666667,69.666667,69.666667,66.000000,66.000000,ingeniería,biotecnología,química,66.000000


Lista final de estudiantes admitidos Química:


,Nombre,Apellido,fac_1,Promedio
0,Kennedy,Barrett,Química,91.333333
1,Ayeshia,Jackman,Química,86.666667
11,Dashanna,Herron,Física,86.666667
12,Laney,Braithwaite,Física,85.333333
14,Genee,Mccrae,Biotecnología,82.333333
13,Blia,Sagar,Física,82.333333
2,Estephanie,Phelps,Química,82.000000
3,Uzma,Naysmythe,Química,81.666667
4,Franki,Dinnis,Química,81.666667
15,Madiha,Milligan,Física,81.666667


# Task
Modify the code in cell "DWLK3KUAzfos" to include the second and third faculty choices (`fac_2` and `fac_3`) for applicants who were not admitted based on their first choice (`fac_1`), prioritizing based on the best required averages per faculty, to finalize the list of admitted students according to the defined quota.

## Identify remaining spots

### Subtask:
Determine how many spots are still available after considering the first choice applicants.


**Reasoning**:
Calculate the remaining number of spots after considering the first-choice applicants.



In [5]:
cupo_disponible = cupo - len(seleccion_fac1)
print(f"Cupo disponible para {facultad.capitalize()}: {cupo_disponible}")

Cupo disponible para Matemáticas: 1


## Filter remaining applicants

### Subtask:
Create a list of applicants who were not selected based on their first choice.


**Reasoning**:
Filter the original dataframe to exclude applicants already selected based on their first choice.



In [7]:
candidatos = df[~df.index.isin(seleccion_fac1.index)].copy()
display(candidatos.head())

,Nombre,Apellido,ESP,EMA,EPH,ECH,ECS,fac_1,fac_2,fac_3,Física_Promedio,Ingeniería_Promedio,Matemáticas_Promedio,Biotecnología_Promedio,Química_Promedio,fac_1_normalized,fac_2_normalized,fac_3_normalized
0,Jermine,Brunton,84,81,73,92,48,Física,Ingeniería,Matemáticas,79.333333,79.333333,79.333333,73.666667,73.666667,física,ingeniería,matemáticas
1,Justo,Mirfin,71,77,61,60,41,Ingeniería,Biotecnología,Química,69.666667,69.666667,69.666667,59.333333,59.333333,ingeniería,biotecnología,química
2,Uzma,Naysmythe,60,94,75,71,80,Química,Ingeniería,Matemáticas,76.333333,76.333333,76.333333,81.666667,81.666667,química,ingeniería,matemáticas
3,Koury,Wingo,71,81,81,83,59,Ingeniería,Biotecnología,Matemáticas,77.666667,77.666667,77.666667,74.333333,74.333333,ingeniería,biotecnología,matemáticas
5,Trica,Macalpine,75,80,96,88,66,Ingeniería,Matemáticas,Biotecnología,83.666667,83.666667,83.666667,78.000000,78.000000,ingeniería,matemáticas,biotecnología


## Process second and third choices

### Subtask:
Iterate through the remaining applicants and check their second and third faculty choices.


**Reasoning**:
Iterate through the remaining applicants and check their second and third faculty choices, calculating their average score if either matches the target faculty.



In [10]:
seleccion_otras = []

for index, row in candidatos.iterrows():
    candidatos_data = row.to_dict()
    candidatos_data['Promedio'] = None # Inicializar promedio

    if row['fac_2_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)
    elif row['fac_3_normalized'] == facultad_lower:
        candidatos_data['Promedio'] = calc_promedio(row, facultad.capitalize())
        seleccion_otras.append(candidatos_data)

# Convertir a DataFrame
seleccion_otras_df = pd.DataFrame(seleccion_otras)
display(seleccion_otras_df.head())

,Nombre,Apellido,ESP,EMA,EPH,ECH,ECS,fac_1,fac_2,fac_3,Física_Promedio,Ingeniería_Promedio,Matemáticas_Promedio,Biotecnología_Promedio,Química_Promedio,fac_1_normalized,fac_2_normalized,fac_3_normalized,Promedio
0,Jermine,Brunton,84,81,73,92,48,Física,Ingeniería,Matemáticas,79.333333,79.333333,79.333333,73.666667,73.666667,física,ingeniería,matemáticas,79.333333
1,Uzma,Naysmythe,60,94,75,71,80,Química,Ingeniería,Matemáticas,76.333333,76.333333,76.333333,81.666667,81.666667,química,ingeniería,matemáticas,76.333333
2,Koury,Wingo,71,81,81,83,59,Ingeniería,Biotecnología,Matemáticas,77.666667,77.666667,77.666667,74.333333,74.333333,ingeniería,biotecnología,matemáticas,77.666667
3,Trica,Macalpine,75,80,96,88,66,Ingeniería,Matemáticas,Biotecnología,83.666667,83.666667,83.666667,78.000000,78.000000,ingeniería,matemáticas,biotecnología,83.666667
4,Sang,Muldoon,84,92,80,60,79,Física,Matemáticas,Ingeniería,85.333333,85.333333,85.333333,77.000000,77.000000,física,matemáticas,ingeniería,85.333333


In [16]:
seleccion_otras_df = seleccion_otras_df.sort_values(by='Promedio', ascending=False)
candidatos_otras = min(cupo_disponible, len(seleccion_otras_df))
seleccion_final_otras = seleccion_otras_df.head(candidatos_otras)
estudiantes_admitidos = pd.concat([seleccion_fac1, seleccion_final_otras], ignore_index=True)

print(f"Lista final de estudiantes admitidos {facultad.capitalize()}:")
lista_final = estudiantes_admitidos.sort_values(by='Promedio', ascending=False)
lista_final[['Nombre', 'Apellido', 'fac_1', 'Promedio']]

Lista final de estudiantes admitidos Matemáticas:


,Nombre,Apellido,fac_1,Promedio
24,Laney,Braithwaite,Física,90.000000
0,Mehul,Bull,Matemáticas,86.000000
1,Spring,Burridge,Matemáticas,84.333333
3,Wynn,Crampton,Matemáticas,84.333333
2,Quinisha,Clarkson,Matemáticas,84.333333
5,Kennon,Inverarity,Matemáticas,84.333333
4,Mir,Ashley,Matemáticas,84.333333
6,Shealynn,Melville,Matemáticas,79.333333
7,Aundria,Guthrie,Matemáticas,78.666667
8,Verlon,Mcconnell,Matemáticas,77.666667
